In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
 
plt.style.use('fivethirtyeight')
%matplotlib inline

# Increase default figure and font sizes for easier viewing.
plt.rcParams['figure.figsize'] = (8, 6)
plt.rcParams['font.size'] = 14

import warnings
warnings.filterwarnings('ignore')


In [ ]:
df = pd.read_csv("Food_Inspections_20250821.csv")

In [2]:
##df = pd.read_csv("Inspections Clean.csv")

In [2]:
# Objectives in lightning talk 2 to start with:
# What are the trends in inspection outcomes over time?
# Which violations are most strongly associated with failure?
# How do different types of food establishments compare in compliance?
# Where are the highest concentrations of failed inspections geographically?
# Why are some high-risk establishments repeatedly failing, and how can they be supported?

# See about adding crime data data set to identify neighbourhoods with higher risk issues. (hasan may have this data might be clean too)
# maybe look at weather data and see if there are any effects. (probably there in the website anyways)
# look into 311 service data to see if there are any complaints.

In [3]:
#--------------------------------------------------------------Cleaning-----------------------------------------------------------------------

In [3]:
df.shape

(252533, 21)

In [5]:
df.head()

Inspection ID                  DBA Name                  AKA Name  \
0        2622439     SMART LEARNING CENTER     SMART LEARNING CENTER   
1        2622414  GENERATION LIQUOR DEPORT  GENERATION LIQUOR DEPORT   
2        2622443           FELD RESTAURANT           FELD RESTAURANT   
3        2622445                   EZBACHI                   EZBACHI   
4        2622418    ROSA ISELA TACOS Y MAS    ROSA ISELA TACOS Y MAS   

   License #                 Facility Type           Risk  \
0  2231591.0  Children's Services Facility  Risk 1 (High)   
1  3020171.0                        Liquor   Risk 3 (Low)   
2  2948639.0                    Restaurant  Risk 1 (High)   
3  3030788.0          Mobile Food Preparer  Risk 1 (High)   
4  2470637.0                    Restaurant  Risk 1 (High)   

                  Address     City State      Zip Inspection Date  \
0  2980-2988 S ARCHER AVE  CHICAGO    IL  60608.0      08/19/2025   
1       3900 W MADISON ST  CHICAGO    IL  60624.0      08/19/2025   
2      2018 W CHICAGO AVE  CHICAGO    IL  60622.0      08/19/2025   
3        135 N KEDZIE AVE  CHICAGO    IL  60612.0      08/19/2025   
4    2551 W FULLERTON AVE  CHICAGO    IL  60647.0      08/19/2025   

         Inspection Type Results  \
0  License Re-Inspection    Pass   
1                License    Pass   
2                Canvass    Pass   
3  License Re-Inspection    Fail   
4                Canvass    Fail   

                                          Violations   Latitude  Longitude  \
0  55. PHYSICAL FACILITIES INSTALLED, MAINTAINED ...  41.840554 -87.660489   
1                                                NaN  41.880865 -87.723376   
2  38. INSECTS, RODENTS, & ANIMALS NOT PRESENT - ...  41.896006 -87.677862   
3  25. CONSUMER ADVISORY PROVIDED FOR RAW/UNDERCO...  41.883905 -87.706059   
4  16. FOOD-CONTACT SURFACES: CLEANED & SANITIZED...  41.924751 -87.691801   

                                   Location  
0   (41.84055377709372, -87.66048949858407)  
1   (41.880865155903734, -87.7233759677506)  
2   (41.89600605332943, -87.67786192982513)  
3   (41.88390509704504, -87.70605925083947)  
4  (41.924751316676854, -87.69180094813183)

In [6]:
df.dtypes

Inspection ID        int64
DBA Name            object
AKA Name            object
License #          float64
Facility Type       object
Risk                object
Address             object
City                object
State               object
Zip                float64
Inspection Date     object
Inspection Type     object
Results             object
Violations          object
Latitude           float64
Longitude          float64
Location            object
dtype: object

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 295508 entries, 0 to 295507
Data columns (total 17 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Inspection ID    295508 non-null  int64  
 1   DBA Name         295508 non-null  object 
 2   AKA Name         293094 non-null  object 
 3   License #        295490 non-null  float64
 4   Facility Type    290266 non-null  object 
 5   Risk             295425 non-null  object 
 6   Address          295508 non-null  object 
 7   City             295350 non-null  object 
 8   State            295450 non-null  object 
 9   Zip              295467 non-null  float64
 10  Inspection Date  295508 non-null  object 
 11  Inspection Type  295507 non-null  object 
 12  Results          295508 non-null  object 
 13  Violations       213298 non-null  object 
 14  Latitude         294497 non-null  float64
 15  Longitude        294497 non-null  float64
 16  Location         294497 non-null  obje

In [8]:
df['Inspection Date'] = pd.to_datetime(df['Inspection Date'])

In [9]:
df['year'] = pd.DatetimeIndex(df['Inspection Date']).year
df['month'] = pd.DatetimeIndex(df['Inspection Date']).month
df['day'] = pd.DatetimeIndex(df['Inspection Date']).day

In [10]:
df['Inspection Type'].value_counts() #Think about this

Inspection Type
Canvass                          153183
License                           39505
Canvass Re-Inspection             33149
Complaint                         27670
License Re-Inspection             12279
                                  ...  
TASK FORCE LIQUOR (1481)              1
LIQOUR TASK FORCE NOT READY           1
POSSIBLE FBI                          1
Kids Cafe'                            1
TASK FORCE PACKAGE GOODS 1474         1
Name: count, Length: 110, dtype: int64

In [11]:
pd.DataFrame(df.isnull().sum())

0
Inspection ID        0
DBA Name             0
AKA Name          2414
License #           18
Facility Type     5242
Risk                83
Address              0
City               158
State               58
Zip                 41
Inspection Date      0
Inspection Type      1
Results              0
Violations       82210
Latitude          1011
Longitude         1011
Location          1011
year                 0
month                0
day                  0

In [12]:
df.dropna(subset=['Facility Type', 'Risk', 'License #', 'Latitude', 'Longitude','Location','Inspection Type'], inplace=True)

In [13]:
df = df.drop(['City', 'State','AKA Name'], axis=1)

In [14]:
df['Results'].value_counts()

Results
Pass                    151755
Fail                     56619
Pass w/ Conditions       44160
Out of Business          20326
No Entry                 12546
Not Ready                 3767
Business Not Located        63
Name: count, dtype: int64

In [15]:
df[(df['License #'] == 0)] # Another form of Nulll?

Inspection ID                            DBA Name  License #  \
1367          2620371       AFC SUSHI @ TONYS FINER FOODS        0.0   
2543          2618821  ANNUNCIATION GREEK ORTHODOX CHURCH        0.0   
2844          2618424              YOUNG SHING FOODS INC.        0.0   
3038          2618140               ASADO COFFEE ROASTERS        0.0   
3141          2617997              EPIPHANY PARISH/CHURCH        0.0   
...               ...                                 ...        ...   
292712         176500                   Oasis Social Club        0.0   
292764          68320                         TACOS REYNA        0.0   
293799         120331                 Unlicensed Facility        0.0   
294275          68074           BY THE HAND CLUB FOR KIDS        0.0   
295399         120273                   QuiteFrankly,Ltd.        0.0   

               Facility Type             Risk               Address      Zip  \
1367              Restaurant    Risk 1 (High)  3607 W FULLERTON AVE  60647.0   
2543                  CHURCH  Risk 2 (Medium)    1017 N LA SALLE DR  60610.0   
2844              Restaurant     Risk 3 (Low)      2119 S PEORIA ST      NaN   
3038              Restaurant  Risk 2 (Medium)     22 E JACKSON BLVD  60604.0   
3141           Special Event  Risk 2 (Medium)     2524 S KEELER AVE  60623.0   
...                      ...              ...                   ...      ...   
292712             religious     Risk 3 (Low)        705 W 79th ST   60620.0   
292764            Restaurant    Risk 1 (High)      3522 E 100TH ST   60617.0   
293799             incubator    Risk 1 (High)     324 N LEAVITT ST   60612.0   
294275  AFTER SCHOOL PROGRAM    Risk 1 (High)     13015 S ELLIS ST   60627.0   
295399            Restaurant    Risk 1 (High)  1400 S JEFFERSON ST   60607.0   

       Inspection Date Inspection Type             Results  \
1367        2025-07-10         Canvass  Pass w/ Conditions   
2543        2025-06-09  Non-Inspection            No Entry   
2844        2025-06-02       Complaint            No Entry   
3038        2025-05-28         Canvass     Out of Business   
3141        2025-05-23         Canvass                Pass   
...                ...             ...                 ...   
292712      2010-03-04       Complaint                Fail   
292764      2010-03-02    Consultation                Fail   
293799      2010-02-08         Canvass  Pass w/ Conditions   
294275      2010-01-28         Canvass                Pass   
295399      2010-01-06         Canvass                Pass   

                                               Violations   Latitude  \
1367    2. CITY OF CHICAGO FOOD SERVICE SANITATION CER...  41.924490   
2543                                                  NaN  41.901081   
2844                                                  NaN  41.853330   
3038                                                  NaN  41.878336   
3141                                                  NaN  41.845143   
...                                                   ...        ...   
292712                                                NaN  41.750566   
292764                                                NaN  41.713977   
293799  1. SOURCE SOUND CONDITION, NO SPOILAGE, FOODS ...  41.887434   
294275                                                NaN  41.658638   
295399  33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSI...  41.863501   

        Longitude                                  Location  year  month  day  
1367   -87.717288   (41.92449009399758, -87.71728813097901)  2025      7   10  
2543   -87.632699   (41.90108084864571, -87.63269935304534)  2025      6    9  
2844   -87.648004   (41.85333023480595, -87.64800350216731)  2025      6    2  
3038   -87.626892  (41.878336071547935, -87.62689227478914)  2025      5   28  
3141   -87.729590   (41.84514342309917, -87.72958950701624)  2025      5   23  
...           ...                                       ...   ...    ...  ...  
292712 -87.6

In [16]:
df[(df['Zip'].isnull())]

Inspection ID                                     DBA Name  License #  \
2844          2618424                       YOUNG SHING FOODS INC.        0.0   
4786          2615962                             BATTER & BERRIES  3020599.0   
36123         2581000                       YOUNG SHING FOODS INC.        0.0   
39402         2577676                            COLORES MEXICANOS  2912277.0   
39808         2577267                            COLORES MEXICANOS  2912277.0   
119107        2222692                        LEO'S FOOD AND LIQUOR  2535068.0   
121371        2200428                        LEO'S FOOD AND LIQUOR  2535068.0   
130824        2135962                  PERSPECTIVES CHARTER SCHOOL  2225546.0   
190281        1538026               FOSTER GROCERY AND DOLLAR PLUS  1992096.0   
194707        1523196                            BURGER KING #1310  2232344.0   
214277        1464217                                DUNKIN DONUTS  1515116.0   
216981        1396156                           THINK SIMPLE FOODS  2308286.0   
217273        1396011                           THINK SIMPLE FOODS  2308286.0   
221238        1228306                              MCCORMICK PLACE  1932251.0   
221302        1228307                              MCCORNICK PLACE  1932251.0   
221341        1375435                        N AND A FOOD MART INC  1980101.0   
224948        1361108   SPORT SERVICE SOLDIER FIELD/SUITE PANTRY D  1354323.0   
224972        1115029                  SPORT SERVICE SOLDIER FIELD  1354323.0   
225039        1115030  SPORT SERVICE SOLDIER FIELD /SUITE PANTRY C  1354323.0   
235639        1326421                             UIC COLLEGE PREP  2225460.0   
235814        1114725                            BURGER KING #1310  2232344.0   
240269        1279336                             UIC COLLEGE PREP  2225460.0   
248164        1134440                       STARBUCKS COFFEE #2901     2328.0   
253953        1106210                                DUNKIN DONUTS  1515116.0   
256259         670661                                DUNKIN DONUTS  1515116.0   
269974         598237   SPORT SERVICE SOLDIER FIELD/SUITE PANTRY D  1354323.0   
270044         598236  SPORT SERVICE SOLDIER FIELD /SUITE PANTRY C  1354323.0   
270096         598235                  SPORT SERVICE SOLDIER FIELD  1354323.0   
275294         525229                      GOLDEN HOUSE RESTAURANT  1443483.0   
276402         391371                      GOLDEN HOUSE RESTAURANT  1443483.0   
277855         483133             JOHNSON COLLEGE PREP HIGH SCHOOL  2353843.0   
293054          63277                                SUN FOOD MART  2009479.0   
295206          67807                                SUN FOOD MART  2009479.0   

        Facility Type             Risk                    Address  Zip  \
2844       Restaurant     Risk 3 (Low)           2119 S PEORIA ST  NaN   
4786       Restaurant    Risk 1 (High)            2318 S STATE ST  NaN   
36123      Restaurant     Risk 3 (Low)           2119 S PEORIA ST  NaN   
39402    RETAIL SALES     Risk 3 (Low)        605 N MICHIGAN AVE   NaN   
39808    RETAIL SALES     Risk 3 (Low)        605 N MICHIGAN AVE   NaN   
119107         Liquor     Risk 3 (Low)       4471 W LAWRENCE AVE   NaN   
121371         Liquor     Risk 3 (Low)       4471 W LAWRENCE AVE   NaN   
130824         School    Risk 1 (High)         1930 S ARCHER AVE   NaN   
190281  Grocery Store     Risk 3 (Low)         3358 W FOSTER AVE   NaN   
194707     Restaurant  Risk 2 (Medium)  2320-2322 S MICHIGAN AVE   NaN   
214277     Restaurant  Risk 2 (Medium)         7545 N PAULINA ST   NaN   
216981     Restaurant  Risk 2 (Medium)         141 W CHICAGO AVE   NaN   
217273     Restaurant  Risk 2 (Medium)         141 W CHICAGO AVE   NaN   
221238     Restaurant    Risk 1 (High)       2301 S INDIANA AVE.   NaN   
221302     Restaurant    Risk 1 (High)            2301 S INDIANA   NaN   
221341  Grocery Store     Risk 3 (Low)         433 E 79TH STREET   Na

In [17]:
df = df[(df["Results"] != "Out of Business") &
        (df["Results"] != "Business Not Located") &
        (df["Results"] != "No Entry") &
        (df["Results"] != "Not Ready")]

In [18]:
df[(df['Violations'].isnull()) & (df['Results'] == 'Fail')] # Maybe remove these

Inspection ID                        DBA Name  License #  \
96            2622245         Maison Pasquale Chicago  3026600.0   
575           2621431  RESTAURANT PRIMO CHUKIS UPTOWN  3040882.0   
608           2621441              LADIES AND PLAYERS  3021252.0   
610           2621445              LADIES AND PLAYERS  3021253.0   
841           2621143            SOUTH SHORE HOSPITAL  2204273.0   
...               ...                             ...        ...   
295371          98317                     BREW & GROW  2013682.0   
295374          58229            FILTER COFFEE LOUNGE  2013495.0   
295436          67756  LAGNIAPPE-A CREOLE CAJUN JOYNT  1986108.0   
295467         158243                   DUNKIN DONUTS  2013344.0   
295488          78240                 OBAMA FOOD MART  2004238.0   

         Facility Type             Risk                  Address      Zip  \
96       Grocery Store     Risk 3 (Low)         2058 W ROSCOE ST  60618.0   
575         Restaurant     Risk 3 (Low)      2721 W LAWRENCE AVE  60625.0   
608         Restaurant     Risk 3 (Low)       4128 N LINCOLN AVE  60618.0   
610         Restaurant     Risk 3 (Low)       4128 N LINCOLN AVE  60618.0   
841           Hospital    Risk 1 (High)       8012 S CRANDON AVE  60617.0   
...                ...              ...                      ...      ...   
295371   Grocery Store     Risk 3 (Low)  3625-3629 N KEDZIE AVE   60618.0   
295374      Restaurant    Risk 1 (High)    1373 N MILWAUKEE AVE   60622.0   
295436  RIVERWALK CAFE  Risk 2 (Medium)    55 W RIVERWALK SOUTH   60601.0   
295467      Restaurant  Risk 2 (Medium)       970 W PERSHING RD   60609.0   
295488   Grocery Store     Risk 3 (Low)          1800 E 79TH ST   60649.0   

       Inspection Date        Inspection Type Results Violations   Latitude  \
96          2025-08-14                License    Fail        NaN  41.943298   
575         2025-07-29                License    Fail        NaN  41.968435   
608         2025-07-29                License    Fail        NaN  41.956772   
610         2025-07-29                License    Fail        NaN  41.956772   
841         2025-07-23                Canvass    Fail        NaN  41.749451   
...                ...                    ...     ...        ...        ...   
295371      2010-01-06           Consultation    Fail        NaN  41.947402   
295374      2010-01-06           Consultation    Fail        NaN  41.906868   
295436      2010-01-05  License Re-Inspection    Fail        NaN  41.886952   
295467      2010-01-05                License    Fail        NaN  41.823504   
295488      2010-01-05                License    Fail        NaN  41.751655   

        Longitude                                 Location  year  month  day  
96     -87.680820  (41.94329829191285, -87.68081954286099)  2025      8   14  
575    -87.697090  (41.96843517181452, -87.69708958435346)  2025      7   29  
608    -87.680757  (41.95677163177934, -87.68075733672262)  2025      7   29  
610    -87.680757  (41.95677163177934, -87.68075733672262)  2025      7   29  
841    -87.568779   (41.74945060417943, -87.5687786244394)  2025      7   23  
...           ...                                      ...   ...    ...  ...  
295371 -87.707695  (41.94740242077963, -87.70769513166378)  2010      1    6  
295374 -87.671749  (41.90686820755232, -87.67174891786233)  2010      1    6  
295436 -87.629875  (41.88695199839048, -87.62987459293426)  2010      1    5  
295467 -87.650666  (41.82350359266238, -87.65066637652201)  2010      1    5  
295488 -87.580828  (41.75165523778611, -87.58082842408824)  2010      1    5  

[3200 rows x 17 columns]

In [19]:
df[(df['Violations'].isnull()) & (df['Results'] == 'Pass w/ Conditions')] # Think about these for a bit.

Inspection ID              DBA Name  License # Facility Type  \
211           2622094        MORDECAI BROWN  3040937.0    Restaurant   
215           2622125       Mendocino Farms  3041410.0    Restaurant   
871           2621069            SHORE CLUB  3031838.0    Restaurant   
881           2621067         EUREST DINING  3036395.0    Restaurant   
895           2621068         EUREST DINING  3031042.0    Restaurant   
...               ...                   ...        ...           ...   
292538         176547           DINER GRILL    57105.0    Restaurant   
292887         176407    MCDONALD' S # 5618  1960250.0    Restaurant   
293083          80249  BROADWAY GRILL, INC.  1620335.0    Restaurant   
293917          68174    MCDONALD' S # 5618  1960250.0    Restaurant   
295269          68255  McDonalds Restaurant    12886.0    Restaurant   

                   Risk                 Address      Zip Inspection Date  \
211        Risk 3 (Low)         3632 N CLARK ST  60613.0      2025-08-12   
215        Risk 3 (Low)          508 N CLARK ST  60654.0      2025-08-12   
871        Risk 3 (Low)    1603 N LAKE SHORE DR  60610.0      2025-07-22   
881        Risk 3 (Low)      401 N MICHIGAN AVE  60611.0      2025-07-22   
895        Risk 3 (Low)      401 N MICHIGAN AVE  60611.0      2025-07-22   
...                 ...                     ...      ...             ...   
292538    Risk 1 (High)  1635 W IRVING PARK RD   60613.0      2010-03-08   
292887  Risk 2 (Medium)        3620 N CLARK ST   60613.0      2010-02-26   
293083    Risk 1 (High)        4800 N BROADWAY   60640.0      2010-02-23   
293917  Risk 2 (Medium)        3620 N CLARK ST   60613.0      2010-02-05   
295269  Risk 2 (Medium)  9211 S COMMERCIAL AVE   60617.0      2010-01-08   

                 Inspection Type             Results Violations   Latitude  \
211                      License  Pass w/ Conditions        NaN  41.948016   
215                      License  Pass w/ Conditions        NaN  41.891150   
871                      License  Pass w/ Conditions        NaN  41.911372   
881                      License  Pass w/ Conditions        NaN  41.889354   
895                      License  Pass w/ Conditions        NaN  41.889354   
...                          ...                 ...        ...        ...   
292538              Consultation  Pass w/ Conditions        NaN  41.954161   
292887              Consultation  Pass w/ Conditions        NaN  41.947712   
293083                 Complaint  Pass w/ Conditions        NaN  41.969083   
293917  Suspected Food Poisoning  Pass w/ Conditions        NaN  41.947712   
295269      Short Form Complaint  Pass w/ Conditions        NaN  41.727880   

        Longitude                                  Location  year  month  day  
211    -87.657340   (41.94801635212034, -87.65733966838366)  2025      8   12  
215    -87.631260  (41.891150496266704, -87.63126028432397)  2025      8   12  
871    -87.626245   (41.91137199562473, -87.62624539660509)  2025      7   22  
881    -87.624102   (41.88935425192819, -87.62410245624348)  2025      7   22  
895    -87.624102   (41.88935425192819, -87.62410245624348)  2025      7   22  
...           ...                                       ...   ...    ...  ...  
292538 -87.670369  (41.954161252677814, -87.67036928186604)  2010      3    8  
292887 -87.657094   (41.94771192275855, -87.65709387286819)  2010      2   26  
293083 -87.659819   (41.96908331914566, -87.65981945361588)  2010      2   23  
293917 -87.657094   (41.94771192275855, -87.65709387286819)  2010      2    5  
295269 -87.551061    (41.72788019121327, -87.5510612280602)  2010      1    8  

[871 rows x 17 columns]

In [20]:
pd.DataFrame(df.isnull().sum())

0
Inspection ID        0
DBA Name             0
License #            0
Facility Type        0
Risk                 0
Address              0
Zip                 19
Inspection Date      0
Inspection Type      0
Results              0
Violations       40979
Latitude             0
Longitude            0
Location             0
year                 0
month                0
day                  0

In [21]:
df[(df['Zip'].isnull())]

Inspection ID                                     DBA Name  License #  \
4786          2615962                             BATTER & BERRIES  3020599.0   
36123         2581000                       YOUNG SHING FOODS INC.        0.0   
39402         2577676                            COLORES MEXICANOS  2912277.0   
39808         2577267                            COLORES MEXICANOS  2912277.0   
119107        2222692                        LEO'S FOOD AND LIQUOR  2535068.0   
216981        1396156                           THINK SIMPLE FOODS  2308286.0   
217273        1396011                           THINK SIMPLE FOODS  2308286.0   
235814        1114725                            BURGER KING #1310  2232344.0   
240269        1279336                             UIC COLLEGE PREP  2225460.0   
253953        1106210                                DUNKIN DONUTS  1515116.0   
256259         670661                                DUNKIN DONUTS  1515116.0   
269974         598237   SPORT SERVICE SOLDIER FIELD/SUITE PANTRY D  1354323.0   
270044         598236  SPORT SERVICE SOLDIER FIELD /SUITE PANTRY C  1354323.0   
270096         598235                  SPORT SERVICE SOLDIER FIELD  1354323.0   
275294         525229                      GOLDEN HOUSE RESTAURANT  1443483.0   
276402         391371                      GOLDEN HOUSE RESTAURANT  1443483.0   
277855         483133             JOHNSON COLLEGE PREP HIGH SCHOOL  2353843.0   
293054          63277                                SUN FOOD MART  2009479.0   
295206          67807                                SUN FOOD MART  2009479.0   

        Facility Type             Risk                    Address  Zip  \
4786       Restaurant    Risk 1 (High)            2318 S STATE ST  NaN   
36123      Restaurant     Risk 3 (Low)           2119 S PEORIA ST  NaN   
39402    RETAIL SALES     Risk 3 (Low)        605 N MICHIGAN AVE   NaN   
39808    RETAIL SALES     Risk 3 (Low)        605 N MICHIGAN AVE   NaN   
119107         Liquor     Risk 3 (Low)       4471 W LAWRENCE AVE   NaN   
216981     Restaurant  Risk 2 (Medium)         141 W CHICAGO AVE   NaN   
217273     Restaurant  Risk 2 (Medium)         141 W CHICAGO AVE   NaN   
235814     Restaurant  Risk 2 (Medium)  2320-2322 S MICHIGAN AVE   NaN   
240269         School    Risk 1 (High)              1231 S DAMEN   NaN   
253953     Restaurant  Risk 2 (Medium)         7545 N PAULINA ST   NaN   
256259     Restaurant  Risk 2 (Medium)          7545 N PAULINA ST  NaN   
269974     Restaurant  Risk 2 (Medium)  1410 S MUSEUM CAMPUS DR.   NaN   
270044     Restaurant  Risk 2 (Medium)      1410 S MUSEUM CAMPUS   NaN   
270096     Restaurant  Risk 2 (Medium)  1410 S MUSEUM CAMPUS DR.   NaN   
275294     Restaurant    Risk 1 (High)      4742-44 N RACINE AVE   NaN   
276402     Restaurant    Risk 1 (High)      4742-44 N RACINE AVE   NaN   
277855         School    Risk 1 (High)         6330 S STEWART ST   NaN   
293054  Grocery Store     Risk 3 (Low)          3601 W CERMAK RD   NaN   
295206  Grocery Store     Risk 3 (Low)          3601 W CERMAK RD   NaN   

       Inspection Date        Inspection Type             Results  \
4786        2025-04-17                License                Pass   
36123       2023-08-30   Short Form Complaint                Fail   
39402       2023-06-22  License Re-Inspection                Pass   
39808       2023-06-13                License                Fail   
119107      2018-09-21                License  Pass w/ Conditions   
216981      2014-02-11  License Re-Inspection                Pass   
217273      2014-02-06                License                Fail   
235814      2013-03-18                License                Pass   
240269      2012-12-18                Canvass                Pass   
253953      2012-04-09  Canvass Re-Inspection                Pass   
256259      2012-02-21              Complaint  Pass w/ Conditions   
269974      2011-05-24                Canvass                Pass   
270044   

In [22]:
df.duplicated().sum()

np.int64(0)

In [23]:
df.shape

(252534, 17)

In [24]:
##df.to_csv('checktype.csv', index=False)

In [25]:
keep = [
    'Canvass',
    'License',
    'Canvass Re-Inspection',
    'Complaint',
    'License Re-Inspection',
    'Complaint Re-Inspection',
    'Short Form Complaint',
    'Suspected Food Poisoning',
    'Consultation',
    'Tag Removal',
    'License-Task Force',
    'Recent Inspection',
    'Task Force Liquor 1475',
    'Suspected Food Poisoning Re-inspection',
    'Complaint-Fire',
    'Short Form Fire-Complaint',
    'OUT OF BUSINESS',
    'NO ENTRY',
    'Special Events (Festivals)',
    'Complaint-Fire Re-inspection',
    'Package Liquor 1474'
]

df['Inspection Type'] = df['Inspection Type'].where(df['Inspection Type'].isin(keep), 'Other')


In [26]:
df['Inspection Type'].value_counts()

Inspection Type
Canvass                                   122715
License                                    34553
Canvass Re-Inspection                      32574
Complaint                                  26084
License Re-Inspection                      12099
Complaint Re-Inspection                    11429
Short Form Complaint                        8620
Suspected Food Poisoning                     964
Consultation                                 651
Tag Removal                                  601
License-Task Force                           596
Recent Inspection                            493
Other                                        277
Task Force Liquor 1475                       250
Suspected Food Poisoning Re-inspection       220
Complaint-Fire                               151
Short Form Fire-Complaint                    104
Special Events (Festivals)                    55
Package Liquor 1474                           43
Complaint-Fire Re-inspection                  43
NO E

In [27]:
df.loc[(df['Violations'].isnull()) & (df['Results'] == 'Pass w/ Conditions'), 'Violations'] = 'Missing Violation Info'

In [28]:
df.loc[(df['Violations'].isnull()) & (df['Results'] == 'Fail'), 'Violations'] = 'Missing Violation Info'

In [29]:
df['Risk'].value_counts() #All?

Risk
Risk 1 (High)      191047
Risk 2 (Medium)     45003
Risk 3 (Low)        16483
All                     1
Name: count, dtype: int64

In [30]:
df = df[df['Risk'] != 'All'] #Dropped that singular row with All.

In [31]:
df.shape

(252533, 17)

In [32]:
#--------------------------------------------------------------Cleaning-----------------------------------------------------------------------

In [1]:
#I used AI here to help  with cleaning Violations becase it was too text heavy and I wasn't sure how to approach it.
#The Violation column had the Violation Code/Violation Description/Comment  in each cell that had a value.
#What I tried to do with AI is have it separate them into those three columns into a new table, then I tried connecting it to
#my dataset in tableau. 

In [ ]:
#df = pd.read_csv("clean.csv")

In [24]:
# import re
# import pandas as pd
# from typing import Optional, Dict, Tuple

# def explode_violations(
#     df: pd.DataFrame,
#     violations_col: str = "Violations",
#     id_col: Optional[str] = None,
#     codebook: Optional[Dict[int, str]] = None,
# ) -> Tuple[pd.DataFrame, pd.DataFrame]:
#     """
#     Parse a packed 'Violations' text column into a tidy table where each row is one violation.
    
#     Parameters
#     ----------
#     df : pd.DataFrame
#         Source inspections dataframe.
#     violations_col : str
#         Column name that contains packed violations text.
#     id_col : Optional[str]
#         Unique inspection identifier column. If None, a sequential ID will be created.
#     codebook : Optional[Dict[int, str]]
#         Optional mapping from numeric violation code -> standardized category/label.
    
#     Returns
#     -------
#     violations_tidy : pd.DataFrame
#         Columns: [Inspection_ID, RowInCell, Violation_Code, Violation_Description, Violation_Comment,
#                   Violation_Label (if codebook provided)]
#     df_with_counts : pd.DataFrame
#         Original df plus 'Violation_Count' (how many items were parsed).
#     """
#     assert violations_col in df.columns, f"Column '{violations_col}' not found."

#     # Make a working copy, ensure string dtype
#     work = df.copy()
#     work[violations_col] = work[violations_col].astype("string")

#     # Create/ensure an inspection ID
#     if id_col is None:
#         work = work.reset_index(drop=False).rename(columns={"index": "Inspection_ID"})
#         id_col = "Inspection_ID"
#     else:
#         assert id_col in work.columns, f"ID column '{id_col}' not found."

#     # 1) Split each cell into separate violation items
#     # Normalize separators to " | " then split
#     sep_norm = work[violations_col].fillna("").str.replace(
#         r"(?:\s*\|\s*|\s*;\s*|\n+)", " | ", regex=True
#     )

#     exploded = (
#         work[[id_col, violations_col]]
#         .assign(Violation_Item=sep_norm)
#         .assign(Violation_Item=lambda s: s["Violation_Item"].str.split(r"\s*\|\s*"))
#         .explode("Violation_Item")
#     )

#     # Drop empties
#     exploded["Violation_Item"] = exploded["Violation_Item"].astype("string").str.strip()
#     exploded = exploded[exploded["Violation_Item"].notna() & (exploded["Violation_Item"] != "")].copy()

#     # Keep original order per cell (RowInCell)
#     exploded["RowInCell"] = exploded.groupby(id_col).cumcount() + 1

#     # 2) Extract Code, Description, Comment from each item
#     # Pattern:  "<code>. <description> Comments: <comment>"
#     # Comments part is optional; description is everything between "code." and "Comments:" (non-greedy).
#     code_pat = re.compile(r"^\s*(\d+)")
#     triplet_pat = re.compile(
#         r"^\s*(?P<code>\d+)\.\s*(?P<desc>.*?)(?:\s*Comments:\s*(?P<comment>.*))?$",
#         flags=re.IGNORECASE | re.DOTALL,
#     )

#     def parse_item(text: str):
#         m = triplet_pat.match(text or "")
#         if m:
#             code = m.group("code")
#             desc = (m.group("desc") or "").strip(" .;\t\r\n")
#             comment = (m.group("comment") or "").strip()
#         else:
#             # Fallback: if no dot/comments structure, try to get a leading code and treat rest as desc
#             m2 = code_pat.match(text or "")
#             code = m2.group(1) if m2 else None
#             remainder = (text or "").strip()
#             # Remove "code." if present
#             if code and remainder.startswith(code):
#                 remainder = remainder[len(code):].lstrip(" .-:)\t")
#             desc, comment = remainder, None
#         # Clean desc a bit
#         desc = re.sub(r"\s+", " ", desc).strip()
#         comment = re.sub(r"\s+", " ", comment).strip() if comment else None
#         return code, desc, comment

#     parsed = exploded["Violation_Item"].apply(parse_item)
#     exploded[["Violation_Code", "Violation_Description", "Violation_Comment"]] = pd.DataFrame(parsed.tolist(), index=exploded.index)

#     # Coerce types
#     exploded["Violation_Code"] = pd.to_numeric(exploded["Violation_Code"], errors="coerce").astype("Int64")

#     # 3) Optional: add standardized label via codebook
#     if codebook:
#         exploded["Violation_Label"] = exploded["Violation_Code"].map(codebook)
#     else:
#         exploded["Violation_Label"] = pd.NA

#     # 4) Prepare outputs
#     violations_tidy = exploded[
#         [id_col, "RowInCell", "Violation_Code", "Violation_Description", "Violation_Comment", "Violation_Label"]
#     ].sort_values([id_col, "RowInCell"], kind="stable").reset_index(drop=True)

#     counts = violations_tidy.groupby(id_col).size().rename("Violation_Count")
#     df_with_counts = work.merge(counts, on=id_col, how="left")
#     df_with_counts["Violation_Count"] = df_with_counts["Violation_Count"].fillna(0).astype(int)

#     return violations_tidy, df_with_counts


In [26]:
# ins = pd.read_csv("clean.csv", low_memory=False)
# codebook = {
#     3: "Improper Food Temperature",
#     10: "Handwashing Sink / Supplies",
#     36: "Rodent Infestation",
#     # ...add as needed
# }

# # 3) Explode to tidy violations
# violations_tidy, ins_with_counts = explode_violations(
#     ins, violations_col="Violations", id_col="Inspection ID", codebook=codebook
# )

# # 4) Save for analysis / Tableau
# violations_tidy.to_csv("violations_tidy.csv", index=False)
# ins_with_counts.to_csv("inspections_with_violation_counts.csv", index=False)

In [14]:
# df = df.assign(Violation_Item=df['Violations'].str.split(r'\s*\|\s*')).explode('Violation_Item')

# df['Violation_Item'] = df['Violation_Item'].str.strip()

In [15]:
# # Extract code
# df['Violation_Code'] = df['Violation_Item'].str.extract(r'^\s*(\d+)')

# # Extract description (before "Comments:")
# df['Violation_Description'] = df['Violation_Item'].str.extract(r'^\s*\d+\.\s*(.*?)\s*Comments:', expand=False)

# # Extract comment (after "Comments:")
# df['Violation_Comment'] = df['Violation_Item'].str.extract(r'Comments:\s*(.*)', expand=False)

# # Convert code to integer
# df['Violation_Code'] = pd.to_numeric(df['Violation_Code'], errors='coerce').astype("Int64")


In [16]:
pd.DataFrame(df.isnull().sum())

0
Inspection ID              0
DBA Name                   0
License #                  0
Facility Type              0
Risk                       0
Address                    0
Zip                       58
Inspection Date            0
Inspection Type            0
Results                    0
Violations             36908
Latitude                   0
Longitude                  0
Location                   0
year                       0
month                      0
day                        0
DBA_Name_Clean           120
DBA_Name_Fuzzy           120
BusinessKey                0
Violation_Item         36908
Violation_Code         40978
Violation_Description  42622
Violation_Comment      42622

In [21]:
df.shape

(997878, 24)

In [22]:
df.to_csv('test.csv', index=False)

In [20]:
df['Violation_Comment'].value_counts()

Violation_Comment
MUST LABEL FOOD STORAGE CONTAINERS WHEN FOOD IS NOT IN ORIGINAL PACKAGE.                                                                                                                                                                                                                                            698
VIOLATION STILL PENDING                                                                                                                                                                                                                                                                                             499
NOTED CERTIFIED FOOD MANAGERS ON DUTY WITH NO ALLERGEN TRAINING CERTIFICATION. INSTRUCTED MANAGEMENT THAT ALL FOOD SERVICE MANAGERS ARE REQUIRED TO HAVE THE ALLERGEN TRAINING CERTIFICATION.                                                                                                                       466
SEE VIOLATION #60                             

In [50]:
cleaned_df.to_csv('test.csv', index=False)

In [ ]:
#------------------------------------------------------------------------------------------------------------------------------------------------